In [1]:
!pip install datasets transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
from evaluate import load

notebook_login()

In [ ]:

from datasets import load_dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator
import shutil
import os
import glob
from collections import defaultdict
import collections
import numpy as np
from tqdm.auto import tqdm

In [4]:
from datasets import load_dataset



langs = ["de","en","es","hi"]

translate_langs = ["de","es","hi"]

langs_test = []

langs_translate_test = []

langs_translate_train = ["de.de","en.en","es.es","hi.hi"]

langs_translate_train_all = []



mlqa = {}



for lang1 in langs:

    for lang2 in langs:

        mlqa[f"{lang1}.{lang2}"] = load_dataset("mlqa", f"mlqa.{lang1}.{lang2}")

        langs_test.append(f"{lang1}.{lang2}")



for lang in translate_langs:

    mlqa[f"translate-train.{lang}"] = load_dataset("mlqa", f"mlqa-translate-train.{lang}")

    mlqa[f"translate-test.{lang}"] = load_dataset("mlqa", f"mlqa-translate-test.{lang}")

    langs_translate_test.append(f"translate-test.{lang}")

    langs_translate_train_all.append(f"translate-train.{lang}")

mlqa.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.9k [00:00<?, ?B/s]

The repository for mlqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating test split:   0%|          | 0/4517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/512 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/10123 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5253 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82451 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10253 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [5]:
mlqa

{'de.de': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 4517
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 512
     })
 }),
 'de.en': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 4517
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 512
     })
 }),
 'de.es': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 1776
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 196
     })
 }),
 'de.hi': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 1430
     })
     validation: Dataset({
         features: ['context', 'question', 'answe

In [10]:
mlqa["translate-train.hi"]["train"][0]

{'context': 'वास्तुशिल्प, स्कूल में एक कैथोलिक चरित्र है । मुख्य इमारत के गोल्ड डोम के ऊपर वर्जिन मैरी की एक गोल्डन प्रतिमा है । तुरंत मुख्य इमारत के सामने और इसका सामना करने के लिए, मसीह की एक तांबे की मूर्ति है, जो लीजेंड venite विज्ञापन में omnes के साथ हथियार है । मुख्य इमारत के बगल में पवित्र दिल का बेसिलिका है । बेसिलिका के तुरंत पीछे, प्रार्थना और प्रतिबिंब का एक marian स्थान है । यह lourdes, फ्रांस में ग्रॉटो की एक प्रतिकृति है, जहां वर्जिन मैरी नकारता ने 1858. में मुख्य ड्राइव के अंत में 1858. में संत बर्नाडेट soubirous को दिखाई दिया (और एक सीधी लाइन में जो 3 मूर्तियों और सोने के डोम से जोड़ता है), मैरी की एक सरल, आधुनिक पत्थर की प्रतिमा है ।',
 'question': 'वर्जिन मैरी ने पर फ्रांस में 1858 में कथित रूप से किस को दिखाई दिया?',
 'answers': {'answer_start': [486], 'text': ['संत बर्नाडेट soubirous']},
 'id': '5733be284776f41900661182'}

In [11]:
mlqa["translate-train.hi"]["validation"][0]

{'context': 'सुपर बाउल 50 2015 के सत्र के लिए राष्ट्रीय फुटबॉल लीग (एनएफएल) के चैंपियन का निर्धारण करने के लिए एक अमेरिकी फुटबॉल खेल था । अमेरिकी फुटबॉल सम्मेलन (एएफसी) चैंपियन डेनवर ब्रोंकोस ने राष्ट्रीय फुटबॉल सम्मेलन (nfc) चैंपियन कैरोलिना पैंथर्स 24-10 को अपना तीसरा सुपर बाउल खिताब अर्जित करने के लिए हराया । यह खेल 7 फरवरी, 2016 को सैन फ्रांसिस्को, कैलिफोर्निया में सैन फ्रांसिस्को खाड़ी क्षेत्र में लेवी के स्टेडियम में खेला गया था । जैसा कि यह 50 वां सुपर बाउल था, लीग ने स्वर्ण वर्षगांठ पर विभिन्न स्वर्ण-थीम्ड पहल के साथ बल दिया, साथ ही रोमन अंकों के साथ प्रत्येक सुपर बाउल गेम को नामकरण करने की परंपरा को अस्थायी रूप से निलंबित कर दिया (जिसके तहत खेल को सुपर बाउल एल के रूप में जाना जाता था), ताकि लोगो को प्रमुखता से अरबी अंक 50. की सुविधा हो सके',
 'question': 'कौन सी एनएफएल टीम ने सुपर बाउल 50 में एएफसी का प्रतिनिधित्व किया?',
 'answers': {'answer_start': [164, 164, 164],
  'text': ['डेनवर ब्रोंकोस', 'डेनवर ब्रोंकोस', 'डेनवर ब्रोंकोस']},
 'id': '56be4db0acb8001400a502ec'}

In [ ]:
max_length = 384 

doc_stride = 128 

In [ ]:
def prepare_train_features(examples):

    
    examples["question"] = [q.lstrip() for q in examples["question"]]


    tokenized_examples = tokenizer(

        examples["question"],

        examples["context"],

        truncation="only_second",

        max_length=max_length,

        stride=doc_stride,

        return_overflowing_tokens=True,

        return_offsets_mapping=True,

        padding="max_length",

    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    offset_mapping = tokenized_examples.pop("offset_mapping")


    tokenized_examples["start_positions"] = []

    tokenized_examples["end_positions"] = []



    for i, offsets in enumerate(offset_mapping):

     
        input_ids = tokenized_examples["input_ids"][i]

        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)


        sample_index = sample_mapping[i]

        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:

            tokenized_examples["start_positions"].append(cls_index)

            tokenized_examples["end_positions"].append(cls_index)

        else:

            start_char = answers["answer_start"][0]

            end_char = start_char + len(answers["text"][0])

            token_start_index = 0

            while sequence_ids[token_start_index] != 1:

                token_start_index += 1


            token_end_index = len(input_ids) - 1

            while sequence_ids[token_end_index] != 1:

                token_end_index -= 1


            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):

                tokenized_examples["start_positions"].append(cls_index)

                tokenized_examples["end_positions"].append(cls_index)

            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:

                    token_start_index += 1

                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:

                    token_end_index -= 1

                tokenized_examples["end_positions"].append(token_end_index + 1)



    return tokenized_examples

In [ ]:
def prepare_validation_features(examples):


    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(

        examples["question"],

        examples["context"],

        truncation="only_second",

        max_length=max_length,

        stride=doc_stride,

        return_overflowing_tokens=True,

        return_offsets_mapping=True,

        padding="max_length",

    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")


    tokenized_examples["example_id"] = []



    for i in range(len(tokenized_examples["input_ids"])):


        sequence_ids = tokenized_examples.sequence_ids(i)

        context_index = 1

        sample_index = sample_mapping[i]

        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [

            (o if sequence_ids[k] == context_index else None)

            for k, o in enumerate(tokenized_examples["offset_mapping"][i])

        ]



    return tokenized_examples

In [15]:
from transformers import DefaultDataCollator



data_collator = DefaultDataCollator()

In [ ]:
from tqdm.auto import tqdm

import collections

import numpy as np



def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):

    all_start_logits, all_end_logits = raw_predictions

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

    features_per_example = collections.defaultdict(list)

    for i, feature in enumerate(features):

        features_per_example[example_id_to_index[feature["example_id"]]].append(i)


    predictions = collections.OrderedDict()


    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")


    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]



        valid_answers = []

        

        context = example["context"]

        for feature_index in feature_indices:

            start_logits = all_start_logits[feature_index]

            end_logits = all_end_logits[feature_index]

            offset_mapping = features[feature_index]["offset_mapping"]


            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()

            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            for start_index in start_indexes:

                for end_index in end_indexes:

                    if (

                        start_index >= len(offset_mapping)

                        or end_index >= len(offset_mapping)

                        or offset_mapping[start_index] is None

                        or offset_mapping[end_index] is None

                    ):

                        continue

                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:

                        continue

                    start_char = offset_mapping[start_index][0]

                    end_char = offset_mapping[end_index][1]

                    valid_answers.append(

                        {

                            "score": start_logits[start_index] + end_logits[end_index],

                            "text": context[start_char: end_char]

                        }

                    )

        

        if len(valid_answers) > 0:

            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]

        else:

            best_answer = {"text": "", "score": 0.0}

        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
from collections import defaultdict

mlqa_prep = defaultdict(dict)

def map_datasets(langs, split, prepare_features):

    for lang in langs:

        mlqa_prep[lang][split] = mlqa[lang][split].map(prepare_features, batched=True, 

                                    remove_columns=mlqa[lang][split].column_names)

In [ ]:
def compute_results(langs, split):

    results = {}

    for lang in langs:

        raw_predictions = trainer.predict(mlqa_prep[lang][split])

        mlqa_prep[lang][split].set_format(type=mlqa_prep[lang][split].format["type"], 

                        columns=list(mlqa_prep[lang][split].features.keys()))


        final_predictions = postprocess_qa_predictions(mlqa[lang][split], mlqa_prep[lang][split], raw_predictions.predictions)

        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

        references = [{"id": ex["id"], "answers": ex["answers"]} for ex in mlqa[lang][split]]

        results[lang] = metric.compute(predictions=formatted_predictions, references=references)

    return results

In [19]:
import pandas as pd

def results_df(results_dict, model):

    F1colname = "F1_" + model

    EMcolname = "EM_" + model

    dict_results = defaultdict(list)

    for lang, scores in results_dict.items():

        dict_results["lang"].append(lang)

        dict_results[F1colname].append(scores['f1'])

        dict_results[EMcolname].append(scores['exact_match'])



    avg_f1 = np.average(dict_results[F1colname])

    avg_em = np.average(dict_results[EMcolname])

    dict_results["lang"].append('avg')

    dict_results[F1colname].append(avg_f1)

    dict_results[EMcolname].append(avg_em)

    df_results = pd.DataFrame(dict_results).round(2)

    return df_results


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer



model_name = "xlm-roberta-base"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)



batch_size = 16


training_args = TrainingArguments(

    output_dir="xlm-roberta-base-squad-hi",

    evaluation_strategy="epoch",

    learning_rate=2e-5,

    per_device_train_batch_size=batch_size,

    per_device_eval_batch_size=batch_size,

    num_train_epochs=3,

    weight_decay=0.01,

    save_strategy="epoch",               
    load_best_model_at_end=True,         
    logging_strategy="epoch"             
)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
lang = ["translate-train.hi"]



split = "train"

map_datasets(lang, split, prepare_train_features)



split = "validation"

map_datasets(lang, split, prepare_train_features)

Map:   0%|          | 0/82451 [00:00<?, ? examples/s]

Map:   0%|          | 0/10253 [00:00<?, ? examples/s]

In [26]:
trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=mlqa_prep["translate-train.hi"]["train"],

    eval_dataset=mlqa_prep["translate-train.hi"]["validation"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

In [27]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=8145, training_loss=2.426437866885359, metrics={'train_runtime': 12412.1598, 'train_samples_per_second': 20.993, 'train_steps_per_second': 0.656, 'total_flos': 5.106417998188954e+16, 'train_loss': 2.426437866885359, 'epoch': 3.0})

In [28]:
metric = load("squad")

In [21]:
# # Delete intermediate checkpoints to reduce storage usage
# for checkpoint in glob.glob("bert-base-multilingual-cased-squad-es/checkpoint-*"):
#     shutil.rmtree(checkpoint, ignore_errors=True)

In [29]:
split = "test"

map_datasets(langs_test, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/1776 [00:00<?, ? examples/s]

Map:   0%|          | 0/1430 [00:00<?, ? examples/s]

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

Map:   0%|          | 0/1430 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [30]:
split = "test"

results_translate_train_hi_xlmr = compute_results(langs_test, split)

print(results_translate_train_hi_xlmr)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 4517 example predictions split into 5274 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

...

Post-processing 1723 example predictions split into 2148 features.


  0%|          | 0/1723 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 41.022802745184855, 'f1': 57.2172078824442}, 'de.en': {'exact_match': 38.0340934248395, 'f1': 54.0259870827274}, 'de.es': {'exact_match': 28.32207207207207, 'f1': 41.022759966029184}, 'de.hi': {'exact_match': 33.35664335664335, 'f1': 47.235074311053225}, 'en.de': {'exact_match': 45.33982731901705, 'f1': 57.678584710713636}, 'en.en': {'exact_match': 57.82571182053494, 'f1': 72.44972107364909}, 'en.es': {'exact_match': 37.76889396535313, 'f1': 50.741780969807685}, 'en.hi': {'exact_match': 44.18462789751932, 'f1': 57.625994221781674}, 'es.de': {'exact_match': 30.743243243243242, 'f1': 48.60146050490661}, 'es.en': {'exact_match': 37.76889396535313, 'f1': 58.684861772461296}, 'es.es': {'exact_match': 40.52922139729678, 'f1': 61.74168687952733}, 'es.hi': {'exact_match': 31.804991294254208, 'f1': 49.44326000080262}, 'hi.de': {'exact_match': 36.57342657342657, 'f1': 52.761396346819666}, 'hi.en': {'exact_match': 38.999593330622204, 'f1': 57.195284839499095}, 'hi.es': {

In [31]:
df_results_translate_train_hi_xlmr = results_df(results_translate_train_hi_xlmr, "Translate-train hi xlmr")

df_results_translate_train_hi_xlmr.to_csv("/kaggle/working/results_translate_train_hi_xlmr.csv")

df_results_translate_train_hi_xlmr

lang  F1_Translate-train hi xlmr  EM_Translate-train hi xlmr
0   de.de                       57.22                       41.02
1   de.en                       54.03                       38.03
2   de.es                       41.02                       28.32
3   de.hi                       47.24                       33.36
4   en.de                       57.68                       45.34
5   en.en                       72.45                       57.83
6   en.es                       50.74                       37.77
7   en.hi                       57.63                       44.18
8   es.de                       48.60                       30.74
9   es.en                       58.68                       37.77
10  es.es                       61.74                       40.53
11  es.hi                       49.44                       31.80
12  hi.de                       52.76                       36.57
13  hi.en                       57.20                       39.00
14  hi.es                       46.52                       30.64
15  hi.hi                       64.00                       45.04
16    avg                       54.81                       38.62

In [32]:
!zip -r /kaggle/working/checkpoint-8145.zip /kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/ (stored 0%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/optimizer.pt (deflated 65%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/training_args.bin (deflated 51%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/special_tokens_map.json (deflated 52%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/scheduler.pt (deflated 56%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/trainer_state.json (deflated 66%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/model.safetensors (deflated 27%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/tokenizer_config.json (deflated 77%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/tokenizer.json (deflated 76%)
  adding: kaggle/working/xlm-roberta-base-squad-hi/checkpoint-8145/rng_state.pth (deflated 25%)
  adding: kaggle/working/x